In [ ]:
# file: random_maxcut.py
# author: Anthony Wilkie

# TODO: find more efficient way to implement scenarios (amplitudes of states for O(log(n))

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from matplotlib import pyplot as plt

import networkx as nx

In [ ]:
def draw_cut(G, pos, bitstring, beamer=False):
    S0 = [node for node in G.nodes if bitstring[node] == "0"]
    S1 = [node for node in G.nodes if bitstring[node] == "1"]

    cut_edges = [edge for edge in G.edges if bitstring[edge[0]] != bitstring[edge[1]]]
    uncut_edges = [edge for edge in G.edges if bitstring[edge[0]] == bitstring[edge[1]]]

    nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='#a99b63')
    #nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='#936846')
    nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='#286d8c')
    if beamer:
        nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='#98c9d3', style='dashdot', alpha=0.5)
        nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, edge_color='#98c9d3', style='solid')
        plt.rc('figure', facecolor='#041017')
    else:
        nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='#041017', style='dashdot', alpha=0.5)
        nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, edge_color='#041017', style='solid')
    nx.draw_networkx_labels(G, pos)

In [ ]:
n = 10
p = 0.3
graph_seed = 18
G = nx.gnp_random_graph(n, p, seed=graph_seed)
m = len(G.edges())

# n = 5
# edges = [(0,1), (0,2), (0,3), (0,4), (1,2), (2,3)]
# m = len(edges)
# G = nx.Graph(edges)

draw_cut(G, nx.spring_layout(G, seed=1), '1'*n, True)
plt.axis('off')
# plt.savefig('/home/vilcius/School/utk/PHYS_642-quantum_information/project/n_5_graph.pdf',
            # bbox_inches='tight',
            # pad_inches = 0,
            # transparent=True)
plt.show()

In [ ]:
def bitstring_to_int(bit_string_sample):
    bit_string = "".join(str(bs) for bs in bit_string_sample)
    return int(bit_string, base=2)

In [ ]:
n_wires = n
dev = qml.device("default.qubit", wires=n, shots=1)

# the gates to randomly choose from
single_gates = [qml.RX, qml.RY, qml.RZ]
double_gates = [qml.IsingXX, qml.IsingYY, qml.IsingZZ, qml.IsingXY]

In [ ]:
C, B = qml.qaoa.maxcut(G)
print(C)

In [ ]:
@qml.qnode(dev)
def random_circuit(gamma, beta, seed=12345, sample=False, probs=False, n_layers=1):
    np.random.seed(seed)

    # initialize state to be an equal superposition
    for i in range(n_wires):
        qml.Hadamard(i)

    # choose m random gates
    for i in range(m):
        # choose if gate is single qubit or two qubit
        qubs = np.random.randint(2)

        if qubs == 0:
            wire = np.random.randint(n_wires)
            op = np.random.choice(single_gates)
            op(beta[0], wires=wire)

        else:
            wire = np.random.choice(n_wires, size=2, replace=False)
            op = np.random.choice(double_gates)
            op(gamma[0], wires=[wire.numpy()[0], wire.numpy()[1]])

    # return samples instead of expectation value
    if sample:
        # measurement phase
        return qml.sample()

    # return probabilities instead of expectation value
    if probs:
        return qml.probs(wires=range(n_wires))

    # Currently use the sum of the Cost Hamiltonians.
    return qml.expval(C)

In [ ]:
@qml.qnode(dev)
def qaoa_circuit(gamma, beta, seed=None, sample=False, probs=False, n_layers=1):
    # initialize state to be an equal superposition
    for i in range(n_wires):
        qml.Hadamard(i)

    # apply unitary layers
    for p in range(n_layers):
        qml.qaoa.cost_layer(gamma[p], C)
        qml.qaoa.mixer_layer(beta[p], B)

    # return samples instead of expectation value
    if sample:
        # measurement phase
        return qml.sample()

    # return probabilities instead of expectation value
    if probs:
        return qml.probs(wires=range(n_wires))

    # Currently use the sum of the Cost Hamiltonians.
    return qml.expval(C)

In [ ]:
# np.random.seed(1248)

def optimize_angles(circuit, seed=None, n_layers=1):
    print("\np={:d}".format(n_layers))

    # initialize the parameters near zero
    init_params = 0.01 * np.random.rand(2, n_layers, requires_grad=True)

    # minimize the negative of the objective function
    def objective(params):
        gammas = params[0]
        betas = params[1]
        neg_obj = circuit(gammas, betas, seed, n_layers=n_layers)
        return neg_obj

    # initialize optimizer: Adagrad works well empirically
    opt = qml.AdagradOptimizer(stepsize=0.5)

    # optimize parameters in objective
    params = init_params
    steps = 50
    for i in range(steps):
        params = opt.step(objective, params)
        if (i + 1) % 5 == 0:
            print("Objective after step {:5d}: {: .7f}".format(i + 1, -objective(params)))

    # sample measured bitstrings 100 times
    bit_strings = []
    n_samples = 1000
    for i in range(0, n_samples):
        bit_strings.append(bitstring_to_int(circuit(params[0], params[1], sample=True, n_layers=n_layers)))

    # print optimal parameters and most frequently sampled bitstring
    counts = np.bincount(np.array(bit_strings))
    most_freq_bit_string = np.argmax(counts)
    print(f"Optimized (gamma, beta) vectors:\n{params[:, :n_layers]}")
    print(f"Most frequently sampled bit string is: {most_freq_bit_string:0{n_wires}b} with probability {(counts[most_freq_bit_string]/n_samples):.4f}")

    return params, bit_strings, most_freq_bit_string

In [ ]:
import matplotlib.pyplot as plt
barcolors = ['#286d8c', '#a99b63', '#936846', '#4d7888']

def graph(bitstrings, beamer):

    if beamer:
        xticks = range(0, 2**(n_wires))
        xtick_labels = list(map(lambda x: format(x, f"0{n_wires}b"), xticks))
        bins = np.arange(0, 2**(n_wires)+1) - 0.5

        plt.figure(figsize=(16, 8))
        plt.rc('font', size=16)
        plt.rc('axes', edgecolor='#98c9d3', labelcolor='#98c9d3', titlecolor='#98c9d3', facecolor='#041017')
        plt.rc('figure', facecolor='#041017')
        plt.rc('savefig', facecolor='#041017')
        plt.rc('xtick',color='#98c9d3')
        plt.rc('ytick',color='#98c9d3')
        plt.rc('legend',labelcolor='#98c9d3', edgecolor='#98c9d3',facecolor=(0,0,0,0))
        plt.title("s-QAOA")
        plt.xlabel("Bitstrings")
        plt.ylabel("Frequency")
        plt.xticks(xticks, xtick_labels, rotation="vertical")
        plt.hist(bitstrings,
                 bins=bins,
                 density=True,
                 color=barcolors[0],
                 edgecolor = "#041017",
                 # label=[f'scenario {i}' for i in range(n_scenarios)]
                 )
        # plt.legend()
        plt.tight_layout()
        # plt.savefig('/home/vilcius/School/utk/PHYS_642-quantum_information/project/maxcut_1_beamer.pdf',
                   # transparent=True)
    else:
        xticks = range(0, 2**(n_wires))
        xtick_labels = list(map(lambda x: format(x, f"0{n_wires}b"), xticks))
        bins = np.arange(0, 2**(n_wires)+1) - 0.5

        plt.figure(figsize=(16, 8))
        plt.rc('font', size=16)
        plt.title("s-QAOA")
        plt.xlabel("Bitstrings")
        plt.ylabel("Frequency")
        plt.xticks(xticks, xtick_labels, rotation="vertical")
        plt.hist(bitstrings,
                 bins=bins,
                 density=True,
                 color=barcolors[0],
                 edgecolor = "#041017",
                 # label=[f'scenario {i}' for i in range(n_scenarios)]
                 )

        plt.legend()
        plt.tight_layout()
        # plt.savefig('/home/vilcius/School/utk/PHYS_642-quantum_information/project/maxcut_1.pdf',
        #            transparent=True)

    plt.show()

In [ ]:
params_rand, bitstrings_rand, most_freq_cut_rand = optimize_angles(random_circuit, n_layers=1)
# graph(bitstrings, beamer=True)
# print(qml.draw(random_circuit)(params[0], params[1]))

In [ ]:
params_qaoa, bitstrings_qaoa, most_freq_cut_qaoa = optimize_angles(qaoa_circuit, n_layers=1)
# graph(bitstrings, beamer=True)
# print(qml.draw(qaoa_circuit)(params[0], params[1]))

In [ ]:
draw_cut(G, nx.spring_layout(G, seed=1), f'{most_freq_cut_rand:0{n_wires}b}', True)
plt.axis('off')
# plt.savefig('/home/vilcius/School/utk/PHYS_642-quantum_information/project/n_5_graph.pdf',
            # bbox_inches='tight',
            # pad_inches = 0,
            # transparent=True)
plt.show()

# graph(bitstrings_rand, beamer=True)

In [ ]:
draw_cut(G, nx.spring_layout(G, seed=1), f'{most_freq_cut_qaoa:0{n_wires}b}', True)
plt.axis('off')
# plt.savefig('/home/vilcius/School/utk/PHYS_642-quantum_information/project/n_5_graph.pdf',
            # bbox_inches='tight',
            # pad_inches = 0,
            # transparent=True)
plt.show()

# graph(bitstrings_qaoa, beamer=True)

In [ ]:
print(random_circuit(params_rand[0],params_rand[1]))
print(random_circuit(params_qaoa[0],params_qaoa[1]))

print(qaoa_circuit(params_rand[0],params_rand[1]))
print(qaoa_circuit(params_qaoa[0],params_qaoa[1]))

print(f'{most_freq_cut_qaoa:0{n_wires}b}')

In [ ]:
probs_qaoa = qaoa_circuit(params_qaoa[0], params_qaoa[1], probs=True)
probs_random = random_circuit(params_rand[0], params_rand[1], probs=True)

graph(probs_qaoa, beamer=True)

In [ ]:
num_circs = 50
random_params = []
random_bitstrings = []
random_most = []
random_expvals = []

# set random seed to generate random seeds
np.random.seed(1)
seeds = np.random.choice(10000, num_circs)

for i in range(num_circs):
    print('------------------------------------------------------------')
    print(f"Random Circuit \# {i+1}")
    params, bitstrings, most = optimize_angles(random_circuit, seed=seeds[i], n_layers=1)
    random_params.append(params)
    random_bitstrings.append(bitstrings)
    random_most.append(most)
    random_expvals.append(random_circuit(params[0], params[1]))

In [ ]:
most_freq_cut_qaoa
print(most_freq_cut_qaoa in random_most)
print(random_most)

for param in random_params:
    print(qaoa_circuit(param[0],param[1]))